In [132]:
from copy import copy
import numpy as np
import re
import os
from typing import Iterable

NUM_EXPERIMENTS = 91
assin_report = \
r"""RTE evaluation
Accuracy	Macro F1
--------	--------
  (.....)%	   (.....)

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  (.....)		              (....)"""

tweetsent_report = \
r"""Acc: (......)
F1. neg: (.....) neu: (.....) post: (.....)
Acc dist min: (......)
Acc dist max: (......)
Acc dist min equal: (......)
Acc dist max equal: (......)"""

rounding_assin = [2, 3, 3, 2]
rounding_tweetsent = [5, 3, 3, 3]

def assin_match(task: str):
    return re.search(assin_report, task)

def tweetsent_match(task: str):
    return re.search(tweetsent_report, task)


def get_metrics(task: str) -> tuple:
    match = assin_match(task) if assin_match(task) else tweetsent_match(task)

    if not match:
        raise ValueError('Task report not in ASSIN or TweetSentBR format')

    metrics = (match.group(i) for i in range(1,5))

    return metrics

def set_metrics(task: str, mean: tuple, std: tuple):
    symbol = '\u00b1'
    rounding =  rounding_assin if assin_match(task) else rounding_tweetsent 
    
    metrics_sample = tuple(get_metrics(task))
    metrics_final = [
                     str(mean[i].round(rounding[i])) + 
                     symbol + 
                     str(std[i].round(rounding[i]))
                     for i in range(0,4)
                     ]

    for idx, metric in enumerate(metrics_sample):
        task = re.sub(metric, metrics_final[idx], task)

    task = re.sub("\n  ", "\n", task)
    task = re.sub(r"	   ([0-9])", r"	\1", task)
    task = re.sub(r"		              ([0-9])", r"	\1", task)

    return task

def get_tasks(report: str) ->  Iterable[str]:
    tasks = re.split('Saving generated .*\n', report)
    tasks = (task for task in tasks if len(task) > 0)
    
    return tasks

filepaths = [f for f in os.listdir() if f.startswith('seed')]

scores_list = list()
for filepath in filepaths:
    seed = re.search(r'seed_(.*).txt', filepath).group(1)

    with open(filepath) as f:
        report = f.read()

    report = re.sub('\n\n\n\n*','\n\n',report)
    tasks = get_tasks(report)
    scores = tuple(map(get_metrics, tasks))
    
    assert len(scores) == NUM_EXPERIMENTS

    scores_list.append(scores)

scores_arr = np.array(scores_lst).astype(np.float32)
scores_mean = np.mean(scores_arr, axis=0).round(decimals=5)
scores_std = np.std(scores_arr, axis=0).round(decimals=5)

sample = copy(report)
sample = re.sub('Saved evaluation:','Evaluation:', sample)
sample = re.sub('report/', '', sample)
sample = re.sub('seed/.*/', '', sample)
sample = re.sub('_eval.txt', '', sample)
sample = re.sub('/', ' ', sample)

tasks = tuple(get_tasks(sample))
tasks_mean = tuple(map(set_metrics, tasks, scores_mean, scores_std))
report_mean = "\n".join(tasks_mean)
print(report_mean)

corpus: assin-ptbr

RTE evaluation
Accuracy	Macro F1
--------	--------
85.17±0.76%	0.548±0.037

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
0.765±0.01	0.34±0.02

Evaluation: st-dnn bert_base assin-ptbr

corpus: assin-ptpt

RTE evaluation
Accuracy	Macro F1
--------	--------
85.54±1.63%	0.716±0.089

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
0.786±0.01	0.56±0.06

Evaluation: st-dnn bert_base assin-ptpt

corpus: assin2

RTE evaluation
Accuracy	Macro F1
--------	--------
87.03±0.61%	0.87±0.006

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
0.82±0.007	0.62±0.05

Evaluation: st-dnn bert_base assin2

corpus: TweetSentBR
Acc: 0.70798±0.0042
F1. neg: 0.697±0.009 neu: 0.585±0.006 post: 0.79±0.002
Acc dist min: 0.2920
Acc dist max: 0.2920
Acc dist min equal: 0.1015
Acc dist max equal: 0.1905

Evaluation: st-dnn bert_base tweetsent


corpus: assin-ptbr

RTE evaluation
Accuracy	Macro F1
--------	---